# Fine-tuning CNNs

В этом ноутбуке научимся файнтюнить CNN, обученную на задаче A для решения другой задачи B. Такой подход также носит название **transfer learning**.

Итого:

1. Рассмотрим задачу B (кошка или собака на изображении).
2. Загрузим CNN обученную решать задачу А (1000-class [ImageNet](http://image-net.org/) classification).
3. Научимся предсказывать классы, используя классическое машинное обучение и признаки, полученные с помощью CNN (CNN feature etractor).
4. Заменим последний слой у CNN и будем тренировать только его для решения задачи B.
5. Натренируем (*файнтюним*) всю сеть для решения задачи B.

### 1. Dogs vs Cats Dataset

Загружаем данные

In [ ]:
import os
from urllib.request import urlretrieve

if not os.path.exists("dogs_vs_cats.train.zip"):
    urlretrieve(
        "https://www.dropbox.com/s/ae1lq6dsfanse76/dogs_vs_cats.train.zip?dl=1",
        "dogs_vs_cats.train.zip")

if not os.path.exists("dogs_vs_cats"):
    import zipfile
    with zipfile.ZipFile("dogs_vs_cats.train.zip", 'r') as archive:
        archive.extractall("dogs_vs_cats")

Делим на train / val и помещаем в отдельные папки

In [ ]:
dataset_root = "dogs_vs_cats"

os.makedirs(os.path.join(dataset_root, "train", "dog"), exist_ok=True)
os.makedirs(os.path.join(dataset_root, "train", "cat"), exist_ok=True)

os.makedirs(os.path.join(dataset_root, "val", "dog"), exist_ok=True)
os.makedirs(os.path.join(dataset_root, "val", "cat"), exist_ok=True)

for filename in os.listdir(os.path.join(dataset_root, "train")):
    if filename.endswith(".jpg"):
        class_name = filename[:3] # "dog" or "cat"
        image_idx = int(filename[4:-4])
        if image_idx % 40 == 0:
            split = "train"
        elif image_idx % 40 == 1:
            split = "val"
        else:
            os.remove(os.path.join(dataset_root, "train", filename))
            continue
        
        os.rename(
            os.path.join(dataset_root, "train", filename),
            os.path.join(dataset_root, split, class_name, filename))

Посмотрим на данные

In [ ]:
import torch
import torchvision

train_dataset = torchvision.datasets.ImageFolder(
    os.path.join(dataset_root, "train"),
    transform=None) # or =torchvision.transforms.ToTensor())

val_dataset = torchvision.datasets.ImageFolder(
    os.path.join(dataset_root, "val"),
    transform=None) # or =torchvision.transforms.ToTensor())

In [ ]:
len(train_dataset), len(val_dataset)

In [ ]:
image, label = train_dataset[100]
print(type(image))
print("cat" if label == 0 else "dog")
print("Image size:", image.size)
image

### 2. CNNs pretrained on the *ImageNet* dataset

#### 2.1 The ImageNet challenge

In [ ]:
import requests

# ImageNet class labels
LABELS_URL = 'https://s3.amazonaws.com/outcome-blog/imagenet/labels.json'
class_names = sorted(list(requests.get(LABELS_URL).json().items()), key=lambda x: int(x[0]))
class_names = list(map(lambda x: x[1], class_names))
n_classes = len(class_names)

In [ ]:
import random

print("Total %d classes; examples:" % n_classes)
for _ in range(10):
    class_idx = random.randint(0, len(class_names))
    print("Class %d: %s" % (class_idx, class_names[class_idx]))

#### 2.2 Pretrained CNNs

In [ ]:
help(torchvision.models)

Для нашего примера воспользуемся сетью ["SqueezeNet"](https://github.com/pytorch/vision/blob/master/torchvision/models/squeezenet.py):

<img src="https://cdn-images-1.medium.com/max/800/1*xji5NAhX6m3Nk7BmR_9GFw.png" width=650>

In [ ]:
?torchvision.models.SqueezeNet

In [ ]:
torchvision.models.SqueezeNet()

In [ ]:
?torchvision.models.squeezenet1_0

In [ ]:
model = torchvision.models.squeezenet1_0(
    pretrained=True,
    num_classes=1000)

Вызваем `.eval()` если хотим делать инференс; если хотим тренировать модель, вызываем `.train()`. Эти методы меняет поведение некоторыз слоев, таких как BatchNorm и Dropout

In [ ]:
model.eval()

In [ ]:
# попробуем запустить модель на случайном наборе данных
# чтобы убедиться, что она не падает
# примечание: SqueezeNet на обучении принимала картинки размером 224x224
# в torchvision до версии 0.4 нужно было убедиться что картинка на вход имеет именно такой размер
# в torchvision версии 0.4 и выше картинка может иметь другой размер
sample_input = torch.randn(5, 3, 224, 224)
sample_output = model(sample_input)

In [ ]:
# the output is class scores:
# (number of images per batch) x (number of classes)
sample_output.shape

In [ ]:
sample_output.min(), sample_output.max()

Попробуем предсказать класс для картинок из `./sample_images/`:

In [ ]:
# See https://pytorch.org/docs/stable/torchvision/models.html
imagenet_mean = torch.tensor([0.485, 0.456, 0.406])
imagenet_std = torch.tensor([0.229, 0.224, 0.225])

In [ ]:
def PIL_to_pytorch(image):
    return torch.tensor(image.getdata()).view(image.size + (3,))

def predict(image):
    """
        image: PIL image
        
        returns: ImageNet class probabilities
    """
    image = PIL_to_pytorch(image)
    image = image.permute(2, 0, 1)
    image = image.to(torch.float32)
    image /= 255.0
    
    # normalize
    image -= imagenet_mean[:, None, None]
    image /= imagenet_std [:, None, None]
    
    # add singleton batch dimension with [None]
    prediction = model(image[None])
    # 1 x 1000
    prediction = prediction.softmax(1)
    
    # remove singleton dimension with [0]
    return prediction[0]

In [ ]:
import PIL.Image

image_path = "sample_Images/albatross.jpg"

image = PIL.Image.open(image_path).resize((224, 224))
image

In [ ]:
prediction = predict(image)

In [ ]:
probabilities = predict(image)

top_probabilities, top_indices = torch.topk(probabilities, 10)
print("10 most probable classes are:")
for probability, class_idx in zip(top_probabilities, top_indices):
    print("%.4f: %s" % (probability, class_names[class_idx]))

### 3. Use classical machine learning with a pretrained CNN

Это последний блок нашей модели (можно убедиться в этом, просмотрев метод `forward()` в [исходниках](https://github.com/pytorch/vision/blob/master/torchvision/models/squeezenet.py):

In [ ]:
model.classifier

Видим, что блок принимает 512-канальное изображение и превращает его в 1000-канальное с помощью линейного слоя (Conv2d с ядром 1x1), и берет среднее значение в каждом канале, чтобы получить логиты итоговых классов.

Давайте из всех слоев блока оставим только average pooling, тем самым модель будет возвращать сырые признаки, которые имеют также название **embeddings**.

In [ ]:
average_pooling = model.classifier[3]

# число "новых классов" 512:
model.num_classes = model.classifier[1].in_channels

model.classifier = average_pooling

In [ ]:
sample_input = torch.randn(5, 3, 224, 224)
sample_embedding = model(sample_input)

In [ ]:
sample_embedding.shape

Эти сырые эмбеддинги уже могут служить хорошими признаками для классификации кот / собака.

Но сначала зададим загрузчики картинок с правильным препроцессингом.

In [ ]:
train_dataset = torchvision.datasets.ImageFolder(
    os.path.join(dataset_root, "train"),
    torchvision.transforms.Compose([
        torchvision.transforms.Resize((224, 224)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=imagenet_mean, std=imagenet_std)]
    )
)

val_dataset = torchvision.datasets.ImageFolder(
    os.path.join(dataset_root, "val"),
    torchvision.transforms.Compose([
        torchvision.transforms.Resize((224, 224)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=imagenet_mean, std=imagenet_std)]
    )
)

In [ ]:
?torch.utils.data.DataLoader

In [ ]:
batch_size = 64

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, num_workers=2, shuffle=True, pin_memory=True)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, num_workers=2, shuffle=True, pin_memory=True)

In [ ]:
# !pip install --user tqdm
from tqdm import tqdm_notebook as tqdm

In [ ]:
import numpy as np

In [ ]:
model.cuda();

Посчитаем эмбеддинги для всего датасета.

In [ ]:
X_train = np.empty((len(train_dataset), sample_embedding.shape[1]), dtype=np.float32)
y_train = np.empty(len(train_dataset), dtype=np.int64)

with torch.no_grad():
    for batch_idx, (image_batch, labels_batch) in enumerate(tqdm(train_loader)):
        embedding_subarray = X_train[batch_idx*batch_size : (batch_idx+1)*batch_size]
        labels_subarray    = y_train[batch_idx*batch_size : (batch_idx+1)*batch_size]
        
        embeddings = model(image_batch.cuda())
        np.copyto(embedding_subarray, embeddings.cpu().numpy())
        np.copyto(labels_subarray, labels_batch.numpy())

In [ ]:
X_val = np.empty((len(val_dataset), sample_embedding.shape[1]), dtype=np.float32)
y_val = np.empty(len(val_dataset), dtype=np.int64)

with torch.no_grad():
    for batch_idx, (image_batch, labels_batch) in enumerate(tqdm(val_loader)):
        embedding_subarray = X_val[batch_idx*batch_size : (batch_idx+1)*batch_size]
        labels_subarray    = y_val[batch_idx*batch_size : (batch_idx+1)*batch_size]
        
        embeddings = model(image_batch.cuda())
        np.copyto(embedding_subarray, embeddings.cpu().numpy())
        np.copyto(labels_subarray, labels_batch.numpy())

In [ ]:
X_train.shape, X_val.shape

In [ ]:
# Тренируем классификатор на этих эмбеддингах
import sklearn.ensemble
import sklearn.metrics

classifier = sklearn.ensemble.RandomForestClassifier()
classifier.fit(X_train, y_train);

In [ ]:
print("Train accuracy: %.2f" % (sklearn.metrics.accuracy_score(y_train, classifier.predict(X_train)) * 100))
print("Val accuracy: %.2f" % (sklearn.metrics.accuracy_score(y_val, classifier.predict(X_val)) * 100))

### 4. Replace the last layer and retrain it

Подход выше не очень удобен так как нам нужно запускать отдельный алгоритм.

Более удобный подход это натренировать новую сетку поверх предобученной CNN.

Тем самым мы получаем **end-to-end** пайплайн: единственная CNN предсказывает именно то что нам нужно.

Здесь мы заменим последний слой сетки на простую лог регрессию (один линейный слой), так что точность не будует сильно выше, чем в случае с random forest. Однако, вы можете сделать более мощный классифкатор, например multilayer perceptron.

In [ ]:
# a custom layer to reshape a tensor into batch of vectors
class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.flatten(1)
    
model.classifier = torch.nn.Sequential(
    average_pooling,
    Flatten(),
    torch.nn.BatchNorm1d(512),
    torch.nn.Linear(512, 1),
).cuda()

model.num_classes = 1

Нам необходимо тренировать только последний слой, так что остальные мы "замораживаем"

In [ ]:
for parameter in model.parameters():
    parameter.requires_grad_(False)

trainable_parameters = list(model.classifier.parameters())
for parameter in trainable_parameters:
    parameter.requires_grad_(True)

Тренируем `model.classifier`:

In [ ]:
optimizer = torch.optim.Adam(trainable_parameters, lr=1e-3, weight_decay=1e-4)
loss = torch.nn.BCEWithLogitsLoss()

In [ ]:
def validate(model, dataloader):
    """ Compute accuracy on a dataset """
    model.eval()
    correct, total = 0, 0
    
    with torch.no_grad():
        for images, labels in dataloader:
            probabilities = model(images.cuda()).cpu().flatten()
            predictions = (probabilities > 0.5).long()

            total += len(labels)
            correct += (predictions == labels).sum().item()
            
    return correct / total

def train(model, dataloader, loss, optimizer):
    """ Train for one epoch """
    model.train()
    correct, total = 0, 0
    total_loss = 0.0
    
    for idx, (images, labels) in enumerate(dataloader):
        probabilities = model(images.cuda()).cpu().flatten()
        
        with torch.no_grad():
            predictions = (probabilities > 0.5).long()
            total += len(labels)
            correct += (predictions == labels).sum().item()
        
        loss_value = loss(probabilities, labels.float())
        total_loss += loss_value.item() * len(labels)
        
        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()
    
    return correct / total, total_loss / total

In [ ]:
for epoch in range(20):
    train_accuracy, train_loss = train(model, train_loader, loss, optimizer)
    val_accuracy = validate(model, val_loader)
    print("Epoch %d, train %.2f%% (loss %.4f), val %.2f%%" \
            % (epoch, train_accuracy * 100, train_loss, val_accuracy * 100))

### 5. Fine-tune the whole network

"Замороженная часть" уже является хорошим feature экстрактором. Однако, она не имеет представление о гашем датасете. Давайте птерь обучим все веса, чтобы получить дополнительную точность. Этот процесс называется **fine-tuning**.

Размораживаем веса:

In [ ]:
for parameter in model.parameters():
    parameter.requires_grad_(True)

Так как оригинальные веса хорошо подобраны, нам нужно использовать менее агрессивный оптимизатор (например SGD с momentum) и гораздо более маленький learning rate

In [ ]:
optimizer = torch.optim.SGD(
    model.parameters(), lr=1e-3, momentum=0.9,
    nesterov=True, weight_decay=1e-4)

In [ ]:
for epoch in range(25):
    train_accuracy, train_loss = train(model, train_loader, loss, optimizer)
    val_accuracy = validate(model, val_loader)
    print("Epoch %d, train %.2f%% (loss %.4f), val %.2f%%" \
            % (epoch, train_accuracy * 100, train_loss, val_accuracy * 100))
    
    # gradually reduce learning rate
    for group in optimizer.param_groups:
        group['lr'] *= 0.9